In [48]:
from files import FileName
import pandas as pd
import great_expectations as ge
from data_reading import DataReading
from files import FileName
from paths import path
from data_reading import DataReading
from files import FileName
import os
import pandas as pd
from data_analysis import Analysis
import string

In [57]:
import os

# Get the current directory as an absolute path
current_directory = os.path.abspath(os.curdir)

# Print or use the current directory string
print("Current directory:", current_directory)


Current directory: c:\Users\balaji.madduri.SENECAGLOBAL\Desktop\Data_Project\src


#Eda analysis 

Country_wise data analysis

In [13]:
#checking for the null values in the country_wise data 
Country_wise=dataframes["country_wise"]
Country_wise.head()

,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
0,Afghanistan,36263,1269,25198,9796,106,10,18,3.50,69.49,5.04,35526,737,2.07,Eastern Mediterranean
1,Albania,4880,144,2745,1991,117,6,63,2.95,56.25,5.25,4171,709,17.00,Europe
2,Algeria,27973,1163,18837,7973,616,8,749,4.16,67.34,6.17,23691,4282,18.07,Africa
3,Andorra,907,52,803,52,10,0,0,5.73,88.53,6.48,884,23,2.60,Europe
4,Angola,950,41,242,667,18,1,0,4.32,25.47,16.94,749,201,26.84,Africa


In [14]:
# checking for the null values
Country_wise.isnull().sum()

Country/Region            0
Confirmed                 0
Deaths                    0
Recovered                 0
Active                    0
New cases                 0
New deaths                0
New recovered             0
Deaths / 100 Cases        0
Recovered / 100 Cases     0
Deaths / 100 Recovered    0
Confirmed last week       0
1 week change             0
1 week % increase         0
WHO Region                0
dtype: int64

In [15]:
#seems like no null values found but the column names needed to be formatted
Country_wise.rename(columns={
    'Country/Region': 'country_region',
    'Confirmed': 'confirmed',
    'Deaths': 'deaths',
    'Recovered': 'recovered',
    'Active': 'active',
    'New cases': 'new_cases',
    'New deaths': 'new_deaths',
    'New recovered': 'new_recovered',
    'Deaths / 100 Cases': 'deaths_per_100_cases',
    'Recovered / 100 Cases': 'recovered_per_100_cases',
    'Deaths / 100 Recovered': 'deaths_per_100_recovered',
    'Confirmed last week': 'confirmed_last_week',
    '1 week change': 'one_week_change',
    '1 week % increase': 'one_week_percent_increase',
    'WHO Region': 'who_region'
}, inplace=True)
Country_wise.columns


Index(['country_region', 'confirmed', 'deaths', 'recovered', 'active',
       'new_cases', 'new_deaths', 'new_recovered', 'deaths_per_100_cases',
       'recovered_per_100_cases', 'deaths_per_100_recovered',
       'confirmed_last_week', 'one_week_change', 'one_week_percent_increase',
       'who_region'],
      dtype='object')

In [16]:
#Validations using the great_expectations
#the country/region column should be unique
Country_wise.expect_column_values_to_be_unique('country_region')

{
  "success": true,
  "result": {
    "element_count": 187,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [17]:
#the deaths should be in between the 0 and the current world population
Country_wise.expect_column_values_to_be_between('deaths',0,81985000)

{
  "success": true,
  "result": {
    "element_count": 187,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [18]:
#validating some of the column values must be greater than or equal to zero


columns_to_check = [
    'confirmed', 'deaths', 'recovered', 'active', 'new_cases', 'new_deaths',
    'new_recovered', 'deaths_per_100_cases', 'recovered_per_100_cases',
    'deaths_per_100_recovered', 'confirmed_last_week', 'one_week_change',
    'one_week_percent_increase'
]
for column in columns_to_check:
  Country_wise.expect_column_values_to_be_between(column, min_value=0)
validation_results=Country_wise.validate()
validation_results


{
  "success": false,
  "results": [
    {
      "success": true,
      "expectation_config": {
        "expectation_type": "expect_column_values_to_be_unique",
        "kwargs": {
          "column": "country_region",
          "result_format": "BASIC"
        },
        "meta": {}
      },
      "result": {
        "element_count": 187,
        "missing_count": 0,
        "missing_percent": 0.0,
        "unexpected_count": 0,
        "unexpected_percent": 0.0,
        "unexpected_percent_total": 0.0,
        "unexpected_percent_nonmissing": 0.0,
        "partial_unexpected_list": []
      },
      "meta": {},
      "exception_info": {
        "raised_exception": false,
        "exception_message": null,
        "exception_traceback": null
      }
    },
    {
      "success": true,
      "expectation_config": {
        "expectation_type": "expect_column_values_to_be_between",
        "kwargs": {
          "column": "deaths",
          "min_value": 0,
          "result_format": "BASIC

In [19]:
# 1 value in the one_week_change has negative value and one value in the one_week_percent_increased
df = pd.DataFrame(Country_wise)
df[Country_wise['one_week_change'] < 0]


,country_region,confirmed,deaths,recovered,active,new_cases,new_deaths,new_recovered,deaths_per_100_cases,recovered_per_100_cases,deaths_per_100_recovered,confirmed_last_week,one_week_change,one_week_percent_increase,who_region
88,Jordan,1176,11,1041,124,8,0,0,0.94,88.52,1.06,1223,-47,-3.84,Eastern Mediterranean


In [20]:
df=pd.DataFrame(Country_wise)
df[Country_wise['one_week_percent_increase'] < 0]

,country_region,confirmed,deaths,recovered,active,new_cases,new_deaths,new_recovered,deaths_per_100_cases,recovered_per_100_cases,deaths_per_100_recovered,confirmed_last_week,one_week_change,one_week_percent_increase,who_region
88,Jordan,1176,11,1041,124,8,0,0,0.94,88.52,1.06,1223,-47,-3.84,Eastern Mediterranean


In [21]:
#dropping this row and updating the df again
Country_wise = pd.DataFrame(Country_wise)
Country_wise=Country_wise.drop(Country_wise[Country_wise.one_week_change < 0].index)
Country_wise.shape



(186, 15)

In [22]:
#checking for all the data types if all of them are in uniform
Country_wise.dtypes

country_region                object
confirmed                      int64
deaths                         int64
recovered                      int64
active                         int64
new_cases                      int64
new_deaths                     int64
new_recovered                  int64
deaths_per_100_cases         float64
recovered_per_100_cases      float64
deaths_per_100_recovered     float64
confirmed_last_week            int64
one_week_change                int64
one_week_percent_increase    float64
who_region                    object
dtype: object

In [23]:
#above who is type object after inspection it is string all values so changin gthe type in to the string
df = pd.DataFrame(Country_wise)
df['who_region'] = df['who_region'].astype('string')
df['country_region']=df['country_region'].astype('string')
print(df.dtypes)


country_region               string[python]
confirmed                             int64
deaths                                int64
recovered                             int64
active                                int64
new_cases                             int64
new_deaths                            int64
new_recovered                         int64
deaths_per_100_cases                float64
recovered_per_100_cases             float64
deaths_per_100_recovered            float64
confirmed_last_week                   int64
one_week_change                       int64
one_week_percent_increase           float64
who_region                   string[python]
dtype: object


In [24]:
#validatin the results
Country_wise = ge.from_pandas(df)
columns_to_check = [
    'country_region',
    'confirmed', 'deaths', 'recovered', 'active', 'new_cases', 'new_deaths',
    'new_recovered', 'deaths_per_100_cases', 'recovered_per_100_cases',
    'deaths_per_100_recovered', 'confirmed_last_week', 'one_week_change',
    'one_week_percent_increase'
]
for column in columns_to_check:
    Country_wise.expect_column_values_to_be_in_type_list(column, ['int', 'float', 'str'])
validation_results = Country_wise.validate()
print(validation_results)

{
  "success": false,
  "results": [
    {
      "success": true,
      "expectation_config": {
        "expectation_type": "expect_column_values_to_be_in_type_list",
        "kwargs": {
          "column": "country_region",
          "type_list": [
            "int",
            "float",
            "str"
          ],
          "result_format": "BASIC"
        },
        "meta": {}
      },
      "result": {
        "observed_value": "str"
      },
      "meta": {},
      "exception_info": {
        "raised_exception": false,
        "exception_message": null,
        "exception_traceback": null
      }
    },
    {
      "success": false,
      "expectation_config": {
        "expectation_type": "expect_column_values_to_be_in_type_list",
        "kwargs": {
          "column": "confirmed",
          "type_list": [
            "int",
            "float",
            "str"
          ],
          "result_format": "BASIC"
        },
        "meta": {}
      },
      "result": {
        "

In [ ]:
# convertin the data frame in to the parquet format
Country_wise.to_parquet('Country_wise.parquet',engine='pyarrow',index=False)